# Logistic Regression Using Food Items Dataset

The goal is to develop a **multi-class classification model** that can predict the dietary category (**More Often**, **In Moderation**, or **Less Often**) of a food item based on its nutritional content.

The original baseline used:
- **Multinomial Logistic Regression (MLR)**
- And its regularized forms: **Ridge (L2)**, **Lasso (L1)**, and **ElasticNet (L1 + L2)**

In this notebook, we aim to enhance the performance of this classification task by:

🔍 **Objectives**:
1. Addressing any **class imbalance** present in the dataset.
2. Performing **advanced data preprocessing** such as normalization, outlier handling, or feature selection.
3. Comparing **modeling strategies**:
   - **Multinomial Logistic Regression** vs.
   - **One-vs-Rest (OvR) Classifiers**
4. Evaluating models using appropriate metrics like **accuracy**, **precision**, **recall**, and **F1-score**.
5. Providing insightful **visualizations** and **interpretations** of the results.

This task is essential in the context of health and nutrition, as automating dietary recommendations based on nutrient data can help guide individuals toward better food choices.


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import zscore
from scipy.stats import zscore


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,roc_auc_score, accuracy_score, precision_recall_fscore_support, precision_score, recall_score
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    precision_recall_curve, average_precision_score,
    roc_curve, auc, RocCurveDisplay,
    PrecisionRecallDisplay
)
from sklearn.preprocessing import label_binarize

import os

import warnings
warnings.filterwarnings("ignore", message=".*use_inf_as_na.*")

### Dataset description 
The dataset consists of various **food items** and their **nutritional properties**, such as fat, saturated fat, cholesterol, sodium, sugars, fiber, and more. Each food item is labeled into one of three dietary categories based on its nutritional profile:

- **More Often**: Generally healthier food items recommended for frequent consumption.
- **In Moderation**: Foods that can be consumed occasionally.
- **Less Often**: Foods with poor nutritional balance, high in fat, sugar, or sodium, and should be limited.

Each row in the dataset represents a single food item, and each column corresponds to a nutritional feature.

In [ ]:
#data = pd.read_csv(r"/kaggle/input/food-items/food_items.csv")

dir_path = '/kaggle/input/food-items-dataset'  # where dataset exists
file_name = 'food_items.csv'
data = pd.read_csv(os.path.join(dir_path, file_name))
data.head()

# Data Exploration

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
food_df = data.copy()

In [ ]:
feature_cols = list(food_df.iloc[:, :-1].columns)
len(feature_cols)

In [ ]:
food_df.describe().T

In [ ]:
food_df[food_df.duplicated()]

In [ ]:
food_df[food_df['Calories'] == 110]

In [ ]:
# Correlation Matrix
numeric_df = food_df[feature_cols]   # as all features are numerical
corr_matrix = numeric_df.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=0.5)
plt.title("Correlation Matrix of Food Features")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


### Features distribution

In [ ]:
# Plot distribution for each numeric feature
plt.figure(figsize=(18, 12))
for i, col in enumerate(feature_cols, 1):  
    plt.subplot(5, 4, i)  
    sns.histplot(food_df[col], kde=True, color='blue', bins=30)  # Histogram with KDE curve
    plt.title(f'Distribution of {col}') 
plt.tight_layout()  
plt.show()  


## **most of features are left_skewed**

### check on outliers

In [ ]:
# check for Outliers
plt.figure(figsize=(18, 12))
for i, col in enumerate(feature_cols, 1):
    plt.subplot(5, 4, i)
    sns.boxplot(data=food_df, y=col, color='lightblue')
    plt.title(f'Boxplot of {col}')

plt.tight_layout()
plt.show()

In [ ]:
# Detect outliers using IQR
outlier_summary = {}

for col in feature_cols:
    Q1 = food_df[col].quantile(0.25)
    Q3 = food_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = food_df[(food_df[col] < lower_bound) | (food_df[col] > upper_bound)]
    outlier_summary[col] = len(outliers)

# display number of outliers per feature
pd.Series(outlier_summary).sort_values(ascending=False)


In [ ]:
pd.Series(outlier_summary).sum()

In [ ]:
# use z-score 
z_scores = food_df[feature_cols].apply(zscore)

# Threshold for 95% confidence (~1.96)
threshold = 1.96

# Count outliers for each column
z_outlier_summary = (abs(z_scores) > threshold).sum()

# Show sorted results
z_outlier_summary.sort_values(ascending=False)

# Data Cleaning

### 1. Removing Duplicates

In [ ]:
# No.of duplicates
food_duplicates = food_df.duplicated().sum()
print(f"Number of duplicate rows: {food_duplicates}")

In [ ]:
# Removing duplicates
food_df = food_df.drop_duplicates()
food_df.shape

In [ ]:
# Resetting index
food_df = food_df.reset_index(drop=True)

In [ ]:
food_df.tail(3)

### 2. handle outliers

In [ ]:
from scipy.stats import zscore
import pandas as pd

def handle_outliers(food_df, columns):
    original_rows = food_df.shape[0]
    print(f"Original number of rows: {original_rows}")

    for col in columns:
        # Compute Z-scores
        z_scores = zscore(food_df[col].dropna())
        z_scores = pd.Series(z_scores, index=food_df[col].dropna().index)

        # Step 1: Remove extreme outliers (|z| > 2.33 → 98% confidence)
        extreme_outliers = z_scores[abs(z_scores) > 2.1].index
        food_df.drop(index=extreme_outliers, inplace=True)

        # Step 2: Impute mild outliers (1.96 < |z| ≤ 2.33) with median
        median_value = food_df[col].median()
        z_scores = zscore(food_df[col].dropna())
        z_scores = pd.Series(z_scores, index=food_df[col].dropna().index)

        mild_outliers = z_scores[(abs(z_scores) > 1.96) & (abs(z_scores) <= 2.1)].index
        food_df.loc[mild_outliers, col] = median_value

    final_rows = food_df.shape[0]
    print(f"Number of rows after removing outliers: {final_rows}")
    print(f"Number of rows removed: {original_rows - final_rows}")

    return food_df


In [ ]:
numeric_cols = food_df.select_dtypes(include='number').columns
food_df = handle_outliers(food_df.copy(), numeric_cols)  # inplace 

In [ ]:
food_df = food_df.reset_index(drop=True)

### 3. check class imbalance 

In [ ]:
food_df['class'].value_counts(normalize=True)

In [ ]:
sns.countplot(x='class', data=food_df)
plt.title("Class Distribution")
plt.show()

#### As we can see from the bar chart above, this dataset has three classes: 
- `In Moderation`, `Less Often`, and `More Often`.
-  The three labels are **imbalanced**.
-  For diabetic patients, most food items are in the `In Moderation` and `Less Often` categories. This makes diabetes diet management very hard, so we could build a machine learning model to help patients choose their food.

#### We have three labels meaning our logistic regression model will be multinomial with three classes.

#### `Multinomial logistic regression` model is different from the `one-vs-rest` binary logistic regression. 
- For `one-vs-rest` schema, you need to train an **independent classifier for each class.**
- For example, you need a `More Often` classifier to differentiate a food item between `More Often` and **Not More Often** (or, `In Moderation` and `Less Often`).

# Data Preprocessing

In [ ]:
# prepare data

X = food_df.drop(columns='class')
y = food_df['class']

In [ ]:
'''X = food_df.drop(columns='class')

pd.set_option("mode.use_inf_as_na", True)
# distribution of each numerical feature
plt.figure(figsize=(20, len(X.columns) * 4))
for i, col in enumerate(X.columns):
    plt.subplot(len(X.columns) // 3 + 1, 3, i + 1)
    sns.histplot(X[col], kde=True, bins=30)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()'''

# noneed already done in eda

### Applying Log Transformation
 - **as all features are left-skewed**

In [ ]:
X_log = X.copy()
X_log = X_log.apply(lambda col: np.log1p(col))

In [ ]:
# Comparing before and after log transformation
def compare_distribution(original, transformed, features, n_cols=3):
    n_rows = (len(features) + n_cols - 1) // n_cols
    plt.figure(figsize=(n_cols * 5, n_rows * 4))

    for i, col in enumerate(features):
        plt.subplot(n_rows, n_cols, i + 1)
        sns.kdeplot(original[col], label='Original', color='blue')
        sns.kdeplot(transformed[col], label='Transformed', color='green')
        plt.title(f"{col}")
        plt.legend()

    plt.tight_layout()
    plt.show()

compare_distribution(X, X_log, X.columns)

### Target Encoding

In [ ]:
label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)

In [ ]:
# observe mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Class Mapping:", label_mapping)

### Data Splitting

In [ ]:
# Train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size = 0.2, random_state=42, stratify=y_encoded)

### SMOTE for imbalance handling 

In [ ]:
# apply SMOTE on training set 

### Preprocessing Pipeline

In [ ]:
log_transformer = FunctionTransformer(np.log1p, validate=True)

pipeline = Pipeline([
    #('log', log_transformer),
    ('scaler', StandardScaler())
])

X_train_scaled = pipeline.fit_transform(X_train)

# transforming X_test
X_test_scaled = pipeline.transform(X_test)


In [ ]:
print(f"Training dataset shape, X_train: {X_train_scaled.shape}, y_train: {y_train.shape}")

In [ ]:
print(f"Testing dataset shape, X_test: {X_test_scaled.shape}, y_test: {y_test.shape}")

# Multinomial Logistic Regression

#### 1. Without Regularization

In [ ]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', class_weight='balanced', max_iter=200, random_state=42)

clf.fit(X_train_scaled, y_train)

# predcitions
y_pred = clf.predict(X_test_scaled)

In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp)
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

In [ ]:
evaluate_metrics(y_test, y_pred)

- ## **slightly lower accuracy with class_weight=balanced, but more realistic**

In [ ]:
# 1. confusion matrix
cf = confusion_matrix(y_test, y_pred)

In [ ]:
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=clf.classes_)
disp.plot()
plt.show()

In [ ]:
# ROC Curves
# Binarize the output
classes = np.unique(y_train)
y_test_bin = label_binarize(y_test, classes=classes)
y_score = clf.predict_proba(X_test_scaled)

# Plot ROC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], label=f"Class {classes[i]} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (multinomial)")
plt.legend()
plt.show()


In [ ]:
# Precision-Recall Curves (per class)
precision = dict()
recall = dict()
avg_precision = dict()

for i in range(len(classes)):
    precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
    avg_precision[i] = average_precision_score(y_test_bin[:, i], y_score[:, i])

plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    plt.plot(recall[i], precision[i], label=f"Class {classes[i]} (AP = {avg_precision[i]:.2f})")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curves")
plt.legend()
plt.show()

In [ ]:
# feature importance (coeff comparision)
# Get coefficient matrix
coef = clf.coef_  # shape: [n_classes, n_features]
feature_names = X.columns if hasattr(X, 'columns') else [f'Feature {i}' for i in range(coef.shape[1])]

# Create a heatmap
plt.figure(figsize=(17, 7))
sns.heatmap(pd.DataFrame(coef, index=classes, columns=feature_names), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Feature Importance (Logistic Regression Coefficients)")
plt.ylabel("Class")
plt.xlabel("Feature")

plt.tight_layout()
plt.show()


### Features Importance

In [ ]:
clf.coef_

#### The `coef_` is a coefficients list with three elements, one element is the actual coefficent for class 0, 1, 2.
- To better analyze the coefficients, use three utility methods to sort and visualize them.

In [ ]:
# Extract and sort feature coefficients
def get_feature_coefs(regression_model, label_index, columns):
    coef_dict = {}
    for coef, feat in zip(regression_model.coef_[label_index, :], columns):
        if abs(coef) >= 0.01:
            coef_dict[feat] = coef
    # Sort coefficients
    coef_dict = {k: v for k, v in sorted(coef_dict.items(), key=lambda item: item[1])}
    return coef_dict

# bar colors based on if value is negative or positive
def get_bar_colors(values):
    color_vals = []
    for val in values:
        if val <= 0:
            color_vals.append('r')
        else:
            color_vals.append('g')
    return color_vals

# visualizing coefficients
def visualize_coefs(coef_dict):
    features = list(coef_dict.keys())
    values = list(coef_dict.values())
    y_pos = np.arange(len(features))
    color_vals = get_bar_colors(values)
    plt.rcdefaults()
    fig, ax = plt.subplots()
    ax.barh(y_pos, values, align='center', color=color_vals)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(features)
    # labels read top-to-bottom
    ax.invert_yaxis()  
    ax.set_xlabel('Feature Coefficients')
    ax.set_title('')
    plt.show()

#### Coefficients for Classes

In [ ]:
# coefficents for Class 1, `Less Often`
coef_dict = get_feature_coefs(clf, 1, feature_cols)
visualize_coefs(coef_dict)

#### Unhealthy nutrients such as `Saturated Fat`, `Sugars`, `Cholesterol`, `Total Fat`, other fats., have high positive coefficients.
- Food items containing unhealthy nutrients will have **higher coeficients** and will be more likely to be categorized in the 'Less Often' class.

In [ ]:
# coefficents for Class 2, `More Often` ==> healthiest options
coef_dict = get_feature_coefs(clf, 2, feature_cols)
visualize_coefs(coef_dict)

#### 2.  `L1-Regularized` Multinomial Logistic Regression

In [ ]:
l2_model = LogisticRegression(
    penalty='l2',
    multi_class='multinomial',
    solver='lbfgs',  # works with l2
    C=0.01,            # inverse of regularization strength (lambda)
    random_state=42, 
    max_iter=1000
)
l2_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_l2 = l2_model.predict(X_test_scaled)

In [ ]:
evaluate_metrics(y_test, y_pred_l2)

In [ ]:
print(classification_report(y_test, y_pred_l2, target_names=label_encoder.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred_l2)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix - Multinomial Logistic Regression")
plt.show()

In [ ]:
# coefficents for Class 1, `Less Often`
coef_dict = get_feature_coefs(clf, 2, feature_cols)
visualize_coefs(coef_dict)

#### 3.  `L1-Regularized` Multinomial Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

l1_model = LogisticRegression(
    penalty='l1',
    solver='saga',               
    multi_class='multinomial',   
    C=0.01,                      
    max_iter=5000,              
    random_state=42
)

l1_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred_l1 = l1_model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_l1))
print(classification_report(y_test, y_pred_l1, target_names=label_encoder.classes_))

In [ ]:
# Which features are used
# non-zero coefficients per class
nonzero_counts = np.sum(l1_model.coef_ != 0, axis=1)
print("Number of non-zero coefficients per class:", nonzero_counts)

In [ ]:
feature_names = X_train.columns  # list of original feature names
coefs = l1_model.coef_           # shape: (n_classes, n_features)

In [ ]:
#class labels (e.g., 'In Moderation', 'Less Often', 'More Often')
class_labels = label_encoder.classes_

# loop over each class
for i, class_label in enumerate(class_labels):
    coef_i = coefs[i]
    nonzero_mask = coef_i != 0
    selected_features = feature_names[nonzero_mask]
    selected_coefs = coef_i[nonzero_mask]

    plt.figure(figsize=(10, 5))
    sns.barplot(x=selected_coefs, y=selected_features, palette='coolwarm')
    plt.title(f"Non-Zero Coefficients for Class: {class_label}")
    plt.xlabel("Coefficient Value")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()

# OVR Logistic Regression

In [ ]:
# Create model with OvR (default for solver='liblinear')
model = LogisticRegression(multi_class='ovr', solver='liblinear',class_weight='balanced', max_iter=200, random_state=42) 
model.fit(X_train_scaled, y_train)

# Predict
y_pred2 = model.predict(X_test_scaled)

In [ ]:
# evalute 
evaluate_metrics(y_test, y_pred2)

In [ ]:
cf = confusion_matrix(y_test, y_pred2)

In [ ]:
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=clf.classes_)
disp.plot()
plt.show()

In [ ]:
# ROC Curves
# Binarize the output
classes = np.unique(y_train)
y_test_bin = label_binarize(y_test, classes=classes)
y_score = clf.predict_proba(X_test_scaled)

# Plot ROC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], label=f"Class {classes[i]} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (OVR)")
plt.legend()
plt.show()

In [ ]:
# Precision-Recall Curves (per class)
precision = dict()
recall = dict()
avg_precision = dict()

for i in range(len(classes)):
    precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
    avg_precision[i] = average_precision_score(y_test_bin[:, i], y_score[:, i])

plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    plt.plot(recall[i], precision[i], label=f"Class {classes[i]} (AP = {avg_precision[i]:.2f})")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curves")
plt.legend()
plt.show()

In [ ]:
# feature importance (coeff comparision)
# Get coefficient matrix
coef = clf.coef_  # shape: [n_classes, n_features]
feature_names = X.columns if hasattr(X, 'columns') else [f'Feature {i}' for i in range(coef.shape[1])]

# Create a heatmap
plt.figure(figsize=(17, 7))
sns.heatmap(pd.DataFrame(coef, index=classes, columns=feature_names), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Feature Importance (Logistic Regression Coefficients)")
plt.ylabel("Class")
plt.xlabel("Feature")

plt.tight_layout()
plt.show()


## OVR gives slightly higher accuracy, but its more complex on large scale 

In [ ]:
print("=== Multinomial ===")
print(classification_report(y_test, y_pred, digits=3))

print("=== One-vs-Rest (OvR) ===")
print(classification_report(y_test, y_pred2, digits=3))

# Optional: ROC-AUC Scores
y_proba_multi = clf.predict_proba(X_test_scaled)
y_proba_ovr = model.predict_proba(X_test_scaled)
classes = np.unique(y_train)
y_test_bin = label_binarize(y_test, classes=classes)

auc_multi = roc_auc_score(y_test_bin, y_proba_multi, average='macro', multi_class='ovr')
auc_ovr = roc_auc_score(y_test_bin, y_proba_ovr, average='macro', multi_class='ovr')

print(f"Multinomial ROC-AUC: {auc_multi:.3f}")
print(f"OvR ROC-AUC:        {auc_ovr:.3f}")


### **Regularization Discussion**
- Regularization was not essential in this case.

- The model converged quickly within the given number of iterations (max_iter=200).

- The feature space was not overly complex or high-dimensional, which reduced the risk of overfitting.

- using default C=1.0 yielded stable and generalizable results, with no need to regularization.




### **Effect of Data Balancing**
- Using class_weight='balanced' improves performance on minority classes.

- should see better recall and F1-score for minority classes, as accuracymight be biased

- Without balancing, the model may predict only majority class due to class imbalance.
